In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
df= pd.read_csv('/content/drive/My Drive/SDP/crime_sdp.csv')

In [ ]:
df

,Unnamed: 0,Description,Genre
0,0,Financial TV host Lee Gates and his producer P...,Crime
1,1,"When Louis Bloom, a con man desperate for work...",Crime
2,2,A girl in a small town forms an unlikely bond ...,Non Crime
3,3,After waking up and discovering that he has un...,Crime
4,4,The savage murders of three young children spa...,Crime
...,...,...,...
295,295,A group of Boston-bred gangsters set up shop i...,Crime
296,296,"With an unmanned, half-mile-long freight train...",Non Crime
297,297,As a police psychologist works to talk down an...,Crime
298,298,When aspiring model Jesse moves to Los Angeles...,Non Crime


In [ ]:
df.drop(["Unnamed: 0"],axis=1,inplace=True)

In [ ]:
df.head()

,Description,Genre
0,Financial TV host Lee Gates and his producer P...,Crime
1,"When Louis Bloom, a con man desperate for work...",Crime
2,A girl in a small town forms an unlikely bond ...,Non Crime
3,After waking up and discovering that he has un...,Crime
4,The savage murders of three young children spa...,Crime


In [ ]:
import nltk
nltk.download()
nltk.download('punkt')

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Hit Enter to continue: 
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat..........

True

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import PorterStemmer
#ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Description'][i])
    review = review.lower()
    review = TextBlob(review)
    review= review.correct()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()

In [ ]:
y=pd.get_dummies(df['Genre'])

In [ ]:
y


,Crime,Non Crime
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
295,1,0
296,0,1
297,1,0
298,0,1


In [ ]:
y=y.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier


from sklearn.naive_bayes import MultinomialNB
crime_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=crime_detect_model.predict(X_test)



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_pred)

In [ ]:
confusion_m

array([[20,  4],
       [18, 18]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
accuracy

0.6333333333333333

In [ ]:
from sklearn import svm

clf = svm.SVC(C=16, kernel='linear', gamma='auto')
clf.fit(X_train, y_train)
clf.predict(X_test)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0], dtype=uint8)

In [ ]:
clf.score(X_test,y_test)

0.4

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

# print(train_x)
gnb.fit(X_train, y_train)
gnb.predict(X_test)
gnb.score(X_test,y_test)

0.6833333333333333

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[0.1,1,8,10,16,32]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 8, 10, 16, 32],
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
from sklearn import svm

clf = svm.SVC(C=1, kernel='rbf', gamma='scale')
clf.fit(X_train, y_train)
clf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
clf.score(X_test,y_test)

0.5666666666666667

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec= DecisionTreeClassifier(random_state=0)

clf_dec.fit(X_train, y_train)

clf_dec.predict(X_test)

clf_dec.score(X_test,y_test)

0.6166666666666667

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log= LogisticRegression(random_state=0)

clf_log.fit(X_train, y_train)

clf_log.predict(X_test)

clf_log.score(X_test,y_test)

0.6333333333333333

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=400, random_state=0)

clf_rf.fit(X_train, y_train)

clf_rf.predict(X_test)

clf_rf.score(X_test,y_test)

0.65

# Using Word Vectors

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=8136e3aadbf50d60706cacd99d23050fa6368c70d907f0aed4c9241890588cb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-inol8gfg/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy

In [ ]:
import en_core_web_md
nlp = en_core_web_md.load()

In [ ]:
train=corpus[:245]
train

['financial host lee gate producer patty put extreme situation irate investor take studio',
 'louis bloom con man desperate work muscle world l crime journalism blur line observer participant become star story',
 'girl small town form unlikely bond recently paralyzed man taking care',
 'waking discovering undergone gender assignment surgery assassin seek find doctor responsible',
 'savage murder three young child spark controversial trial three teenager accused killing kiss part tetanic ritual',
 'high wire artist philippe petit recruit team people help realize dream walk immense void world trade center tower',
 'woman look back past nineteen men relationship life wonder one might one true love',
 'identical twin gangster donald regional gray terrorist london',
 'steve job take u behind scene digital revolution paint portrait man epicenter story unfold backstage three ironic product launched ending unveiling mac',
 'story michael homeless dramatized boy became american football player 

In [ ]:
train_x=corpus[:245]
train_y=df.Genre[:245]
test_x=corpus[245:]
test_y=df.Genre[245:]

In [ ]:
print(train_x)
print(train_y)

print(train_x[0])
print(train_y[0])

print(test_x)
print(test_y)

['financial host lee gate producer patty put extreme situation irate investor take studio', 'louis bloom con man desperate work muscle world l crime journalism blur line observer participant become star story', 'girl small town form unlikely bond recently paralyzed man taking care', 'waking discovering undergone gender assignment surgery assassin seek find doctor responsible', 'savage murder three young child spark controversial trial three teenager accused killing kiss part tetanic ritual', 'high wire artist philippe petit recruit team people help realize dream walk immense void world trade center tower', 'woman look back past nineteen men relationship life wonder one might one true love', 'identical twin gangster donald regional gray terrorist london', 'steve job take u behind scene digital revolution paint portrait man epicenter story unfold backstage three ironic product launched ending unveiling mac', 'story michael homeless dramatized boy became american football player first rou

In [ ]:
train_X=train_x
train_Y=train_y
test_X=test_x
test_Y=test_y

In [ ]:
print(type(train_X))

<class 'list'>


In [ ]:
docs = [nlp(text) for text in train_X]
train_x_word_vectors = [x.vector for x in docs]

In [ ]:
from sklearn import svm

clf_svm_wv = svm.SVC(kernel='linear')
clf_svm_wv.fit(train_x_word_vectors, train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
test_docs = [nlp(text) for text in test_X]
test_x_word_vectors =  [x.vector for x in test_docs]

clf_svm_wv.predict(test_x_word_vectors)

array(['Crime', 'Crime', 'Crime', 'Crime', 'Non Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Crime', 'Non Crime', 'Crime',
       'Crime', 'Crime', 'Crime', 'Non Crime', 'Crime', 'Non Crime',
       'Crime', 'Crime', 'Crime', 'Crime', 'Non Crime', 'Non Crime',
       'Non Crime', 'Crime', 'Crime', 'Crime', 'Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Crime', 'Non Crime', 'Crime',
       'Crime', 'Non Crime', 'Crime', 'Non Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Non Crime', 'Crime',
       'Non Crime', 'Crime', 'Non Crime', 'Crime', 'Non Crime',
       'Non Crime'], dtype=object)

In [ ]:
clf_svm_wv.score(test_x_word_vectors,test_Y)

0.7818181818181819

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec_wv= DecisionTreeClassifier(random_state=0)

clf_dec_wv.fit(train_x_word_vectors, train_Y)

clf_dec_wv.predict(test_x_word_vectors)

clf_dec_wv.score(test_x_word_vectors,test_Y)

0.6363636363636364

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_wv = GaussianNB()

# print(train_x)
gnb_wv.fit(train_x_word_vectors, train_Y)
gnb_wv.predict(test_x_word_vectors)
gnb_wv.score(test_x_word_vectors,test_Y)

0.8181818181818182

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log_wv= LogisticRegression(random_state=0)

clf_log_wv.fit(train_x_word_vectors, train_Y)

clf_log_wv.predict(test_x_word_vectors)

clf_log_wv.score(test_x_word_vectors,test_Y)

0.8363636363636363

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf_wv = RandomForestClassifier(n_estimators=400, random_state=0)

clf_rf_wv.fit(train_x_word_vectors, train_Y)

clf_rf_wv.predict(test_x_word_vectors)

clf_rf_wv.score(test_x_word_vectors,test_Y)

0.8363636363636363

In [ ]:
corpus

['financial host lee gate producer patty put extreme situation irate investor take studio',
 'louis bloom con man desperate work muscle world l crime journalism blur line observer participant become star story',
 'girl small town form unlikely bond recently paralyzed man taking care',
 'waking discovering undergone gender assignment surgery assassin seek find doctor responsible',
 'savage murder three young child spark controversial trial three teenager accused killing kiss part tetanic ritual',
 'high wire artist philippe petit recruit team people help realize dream walk immense void world trade center tower',
 'woman look back past nineteen men relationship life wonder one might one true love',
 'identical twin gangster donald regional gray terrorist london',
 'steve job take u behind scene digital revolution paint portrait man epicenter story unfold backstage three ironic product launched ending unveiling mac',
 'story michael homeless dramatized boy became american football player 

# USING RNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=10000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4563, 9807, 967, 6411, 6526, 5840, 5737, 3687, 7468, 3981, 631, 1655, 1696],
 [8918,
  1926,
  8321,
  5227,
  8700,
  509,
  2696,
  1343,
  6700,
  6596,
  2055,
  6185,
  2967,
  3786,
  8630,
  1315,
  4884,
  2256],
 [2223, 6097, 1406, 2108, 716, 450, 6605, 2816, 5227, 6961, 7653],
 [4325, 6443, 202, 7963, 7864, 6527, 2733, 6872, 924, 3207, 4065],
 [3038,
  6700,
  4632,
  8272,
  9463,
  186,
  259,
  9981,
  4632,
  7978,
  2970,
  8919,
  3226,
  8665,
  9203,
  2160],
 [7880,
  9998,
  8147,
  5688,
  3131,
  1135,
  439,
  8250,
  4598,
  4676,
  8117,
  731,
  8728,
  6640,
  1343,
  757,
  3539,
  1383],
 [3828,
  1588,
  6735,
  5379,
  6960,
  1469,
  1587,
  7349,
  5990,
  1706,
  9761,
  1706,
  903,
  1898],
 [361, 5930, 5486, 8672, 5947, 2572, 7227, 5329],
 [1433,
  8814,
  1655,
  4119,
  572,
  1742,
  4088,
  772,
  730,
  7785,
  5227,
  2476,
  2256,
  5858,
  6673,
  4632,
  9696,
  9034,
  5380,
  401,
  9792,
  803],
 [2256,
  1450,
  6102,
  3909,
  7044,


In [ ]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  631 1655 1696]
 [   0    0    0 ... 1315 4884 2256]
 [   0    0    0 ... 5227 6961 7653]
 ...
 [   0    0    0 ... 1264  768 5404]
 [   0    0    0 ... 6557 7759 6825]
 [   0    0    0 ... 9906 6602 5659]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4563, 9807,  967, 6411, 6526, 5840, 5737,
       3687, 7468, 3981,  631, 1655, 1696], dtype=int32)

In [ ]:
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SimpleRNN(128,return_sequences=True))
model.add(SimpleRNN(128,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 50, 128)           29312     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,062,337
Trainable params: 1,062,337
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 71ms/step - loss: 2.2650e-04 - accuracy: 1.0000 - val_loss: 1.2473 - val_accuracy: 0.5333
Epoch 2/10
8/8 [==============================] - 1s 71ms/step - loss: 2.1773e-04 - accuracy: 1.0000 - val_loss: 1.2523 - val_accuracy: 0.5333
Epoch 3/10
8/8 [==============================] - 1s 74ms/step - loss: 2.0962e-04 - accuracy: 1.0000 - val_loss: 1.2531 - val_accuracy: 0.5333
Epoch 4/10
8/8 [==============================] - 1s 68ms/step - loss: 2.0141e-04 - accuracy: 1.0000 - val_loss: 1.2595 - val_accuracy: 0.5333
Epoch 5/10
8/8 [==============================] - 1s 72ms/step - loss: 1.9513e-04 - accuracy: 1.0000 - val_loss: 1.2612 - val_accuracy: 0.5333
Epoch 6/10
8/8 [==============================] - 1s 76ms/step - loss: 1.8822e-04 - accuracy: 1.0000 - val_loss: 1.2663 - val_accuracy: 0.5333
Epoch 7/10
8/8 [==============================] - 1s 77ms/step - loss: 1.8176e-04 - accuracy: 1.0000 - val_loss: 1.2702 - val_accuracy: 0.5333

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=100
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(SimpleRNN(64))
#model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20
8/8 [==============================] - 1s 64ms/step - loss: 0.6979 - accuracy: 0.5378 - val_loss: 0.7064 - val_accuracy: 0.4267
Epoch 2/20
8/8 [==============================] - 0s 41ms/step - loss: 0.6015 - accuracy: 0.7067 - val_loss: 0.7195 - val_accuracy: 0.4400
Epoch 3/20
8/8 [==============================] - 0s 43ms/step - loss: 0.4697 - accuracy: 0.9556 - val_loss: 0.7029 - val_accuracy: 0.4933
Epoch 4/20
8/8 [==============================] - 0s 42ms/step - loss: 0.3078 - accuracy: 0.9911 - val_loss: 0.7228 - val_accuracy: 0.4933
Epoch 5/20
8/8 [==============================] - 0s 42ms/step - loss: 0.1639 - accuracy: 0.9956 - val_loss: 0.7416 - val_accuracy: 0.5067
Epoch 6/20
8/8 [==============================] - 0s 42ms/step - loss: 0.0703 - accuracy: 1.0000 - val_loss: 0.7838 - val_accuracy: 0.5200
Epoch 7/20
8/8 [==============================] - 0s 42ms/step - loss: 0.0390 - accuracy: 1.0000 - val_loss: 0.8411 - val_accuracy: 0.5200
Epoch 8/20
8/8 [===========

In [ ]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5333333333333333

## Using LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=10000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4563, 9807, 967, 6411, 6526, 5840, 5737, 3687, 7468, 3981, 631, 1655, 1696],
 [8918,
  1926,
  8321,
  5227,
  8700,
  509,
  2696,
  1343,
  6700,
  6596,
  2055,
  6185,
  2967,
  3786,
  8630,
  1315,
  4884,
  2256],
 [2223, 6097, 1406, 2108, 716, 450, 6605, 2816, 5227, 6961, 7653],
 [4325, 6443, 202, 7963, 7864, 6527, 2733, 6872, 924, 3207, 4065],
 [3038,
  6700,
  4632,
  8272,
  9463,
  186,
  259,
  9981,
  4632,
  7978,
  2970,
  8919,
  3226,
  8665,
  9203,
  2160],
 [7880,
  9998,
  8147,
  5688,
  3131,
  1135,
  439,
  8250,
  4598,
  4676,
  8117,
  731,
  8728,
  6640,
  1343,
  757,
  3539,
  1383],
 [3828,
  1588,
  6735,
  5379,
  6960,
  1469,
  1587,
  7349,
  5990,
  1706,
  9761,
  1706,
  903,
  1898],
 [361, 5930, 5486, 8672, 5947, 2572, 7227, 5329],
 [1433,
  8814,
  1655,
  4119,
  572,
  1742,
  4088,
  772,
  730,
  7785,
  5227,
  2476,
  2256,
  5858,
  6673,
  4632,
  9696,
  9034,
  5380,
  401,
  9792,
  803],
 [2256,
  1450,
  6102,
  3909,
  7044,


In [ ]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)


[[   0    0    0 ...  631 1655 1696]
 [   0    0    0 ... 1315 4884 2256]
 [   0    0    0 ... 5227 6961 7653]
 ...
 [   0    0    0 ... 1264  768 5404]
 [   0    0    0 ... 6557 7759 6825]
 [   0    0    0 ... 9906 6602 5659]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4563, 9807,  967, 6411, 6526, 5840, 5737,
       3687, 7468, 3981,  631, 1655, 1696], dtype=int32)

In [ ]:
embedding_vector_features=300
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 3,160,501
Trainable params: 3,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(300, (300,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((300, 50), (300,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20
8/8 [==============================] - 0s 35ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.2499 - val_accuracy: 0.6800
Epoch 2/20
8/8 [==============================] - 0s 33ms/step - loss: 9.3907e-04 - accuracy: 1.0000 - val_loss: 1.2719 - val_accuracy: 0.7067
Epoch 3/20
8/8 [==============================] - 0s 34ms/step - loss: 8.3497e-04 - accuracy: 1.0000 - val_loss: 1.3127 - val_accuracy: 0.6933
Epoch 4/20
8/8 [==============================] - 0s 33ms/step - loss: 7.6149e-04 - accuracy: 1.0000 - val_loss: 1.3797 - val_accuracy: 0.7067
Epoch 5/20
8/8 [==============================] - 0s 33ms/step - loss: 6.8045e-04 - accuracy: 1.0000 - val_loss: 1.3807 - val_accuracy: 0.7067
Epoch 6/20
8/8 [==============================] - 0s 34ms/step - loss: 6.1335e-04 - accuracy: 1.0000 - val_loss: 1.3863 - val_accuracy: 0.7067
Epoch 7/20
8/8 [==============================] - 0s 32ms/step - loss: 5.6481e-04 - accuracy: 1.0000 - val_loss: 1.4024 - val_accuracy: 0.7200
Epo

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=500
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.5))
model1.add(LSTM(100))
#model.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:

model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20
8/8 [==============================] - 0s 55ms/step - loss: 5.7936e-04 - accuracy: 1.0000 - val_loss: 1.3427 - val_accuracy: 0.6933
Epoch 2/20
8/8 [==============================] - 0s 51ms/step - loss: 4.3399e-04 - accuracy: 1.0000 - val_loss: 1.3715 - val_accuracy: 0.6933
Epoch 3/20
8/8 [==============================] - 0s 52ms/step - loss: 4.1996e-04 - accuracy: 1.0000 - val_loss: 1.3982 - val_accuracy: 0.6933
Epoch 4/20
8/8 [==============================] - 0s 52ms/step - loss: 4.1755e-04 - accuracy: 1.0000 - val_loss: 1.4263 - val_accuracy: 0.6933
Epoch 5/20
8/8 [==============================] - 0s 52ms/step - loss: 4.2203e-04 - accuracy: 1.0000 - val_loss: 1.4646 - val_accuracy: 0.7067
Epoch 6/20
8/8 [==============================] - 0s 52ms/step - loss: 3.2469e-04 - accuracy: 1.0000 - val_loss: 1.5019 - val_accuracy: 0.7067
Epoch 7/20
8/8 [==============================] - 0s 53ms/step - loss: 3.2376e-04 - accuracy: 1.0000 - val_loss: 1.5303 - val_accuracy: 0.7067

In [ ]:
y_pred1=model1.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[32,  9],
       [11, 23]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.7333333333333333

# Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
## Creating model
embedding_vector_features=500
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(Bidirectional(LSTM(100)))
#model.add(Dropout(0.5))
model2.add(Dense(1,activation='sigmoid'))

model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 50, 500)           5000000   
_________________________________________________________________
dropout_10 (Dropout)         (None, 50, 500)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 200)               480800    
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 201       
Total params: 5,481,001
Trainable params: 5,481,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15)

Epoch 1/15
8/8 [==============================] - 0s 58ms/step - loss: 3.7449e-04 - accuracy: 1.0000 - val_loss: 1.4652 - val_accuracy: 0.6667
Epoch 2/15
8/8 [==============================] - 0s 59ms/step - loss: 3.5911e-04 - accuracy: 1.0000 - val_loss: 1.4865 - val_accuracy: 0.6667
Epoch 3/15
8/8 [==============================] - 0s 57ms/step - loss: 3.3877e-04 - accuracy: 1.0000 - val_loss: 1.5082 - val_accuracy: 0.6667
Epoch 4/15
8/8 [==============================] - 0s 55ms/step - loss: 3.2650e-04 - accuracy: 1.0000 - val_loss: 1.5313 - val_accuracy: 0.6667
Epoch 5/15
8/8 [==============================] - 0s 55ms/step - loss: 3.0917e-04 - accuracy: 1.0000 - val_loss: 1.5537 - val_accuracy: 0.6533
Epoch 6/15
8/8 [==============================] - 0s 53ms/step - loss: 2.9358e-04 - accuracy: 1.0000 - val_loss: 1.5774 - val_accuracy: 0.6667
Epoch 7/15
8/8 [==============================] - 0s 56ms/step - loss: 2.8473e-04 - accuracy: 1.0000 - val_loss: 1.6048 - val_accuracy: 0.6667

In [ ]:
y_pred2=model2.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred2)

array([[28, 13],
       [ 9, 25]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.7066666666666667

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72        41
           1       0.66      0.74      0.69        34

    accuracy                           0.71        75
   macro avg       0.71      0.71      0.71        75
weighted avg       0.71      0.71      0.71        75



# THE END